In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
books = pd.read_csv(r'C:\Users\hi\Desktop\Book Remainder System\Books.csv')
ratings = pd.read_csv(r'C:\Users\hi\Desktop\Book Remainder System\Ratings.csv')
users = pd.read_csv(r'C:\Users\hi\Desktop\Book Remainder System\Users.csv')

C:\Users\hi\AppData\Local\Temp\ipykernel_3580\285858933.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(r'C:\Users\hi\Desktop\Book Remainder System\Books.csv')


# Popularity Based Recomendation System

In [5]:
rwn = ratings.merge(books,on='ISBN')
num_rating_df = rwn.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num-ratings'},inplace=True)
num_rating_df
num_rating_df['num-ratings'].isnull().sum()

np.int64(0)

In [6]:
# Drop rows where 'Book-Rating' is NaN
rwn = rwn.dropna(subset=['Book-Rating'])

# Now group and calculate the mean again
avg_rating_df = rwn.groupby('Book-Title', as_index=False).agg({'Book-Rating': 'mean'})

# Rename the column
avg_rating_df.rename(columns={'Book-Rating': 'avg-ratings'}, inplace=True)

# Display the dataframe
print(avg_rating_df)


                                               Book-Title  avg-ratings
0        A Light in the Storm: The Civil War Diary of ...     2.250000
1                                   Always Have Popsicles     0.000000
2                    Apple Magic (The Collector's series)     0.000000
3        Ask Lily (Young Women of Faith: Lily Series, ...     8.000000
4        Beyond IBM: Leadership Marketing and Finance ...     0.000000
...                                                   ...          ...
241066                                      Ã?Â?lpiraten.     0.000000
241067                     Ã?Â?rger mit Produkt X. Roman.     5.250000
241068                                Ã?Â?sterlich leben.     7.000000
241069                              Ã?Â?stlich der Berge.     2.666667
241070                                  Ã?Â?thique en toc     4.000000

[241071 rows x 2 columns]


In [7]:
popularity_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popularity_df

,Book-Title,num-ratings,avg-ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [8]:
popularity_df = popularity_df[popularity_df['num-ratings']>250].sort_values('avg-ratings',ascending=False).head(50)

In [9]:
popular_df = popularity_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num-ratings','avg-ratings']]

In [10]:
popular_df['Image-URL-M'][0]

'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'

# Colabrative Filtering

In [11]:
X = rwn.groupby('User-ID').count()['Book-Rating'] >200 # It shows the number of ratings for each user.
padhe_likhe_users = X[X].index

In [12]:
filtered_rating = rwn[rwn['User-ID'].isin(padhe_likhe_users)]

In [13]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index
famous_books
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [14]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(pt)

In [31]:
def recommendation(book_name):
    # Index Fetch
    index = np.where(pt.index==book_name)[0][0]
    similer_items = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:6]
    data = []
    for i in similer_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

        data.append(item)
    return data    

In [32]:
recommendation('Harry Potter and the Chamber of Secrets (Book 2)')

[['Harry Potter and the Prisoner of Azkaban (Book 3)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Goblet of Fire (Book 4)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg'],
 ["Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
  'J. K. Rowling',
  'http://images.amazon.com/images/P/059035342X.01.MZZZZZZZ.jpg'],
 ["Harry Potter and the Sorcerer's Stone (Book 1)",
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0590353403.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Order of the Phoenix (Book 5)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/043935806X.01.MZZZZZZZ.jpg']]

In [18]:
pt.index[545]

"The Handmaid's Tale"

In [19]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [20]:
popular_df.head()

,Book-Title,Book-Author,Image-URL-M,num-ratings,avg-ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453


In [21]:
popular_df['Image-URL-M'].iloc[22]

'http://images.amazon.com/images/P/0805063897.01.MZZZZZZZ.jpg'

In [33]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','b'))
pickle.dump(similarity_score,open('similarity_scores.pkl','wb'))